In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

import sqlite3

from IPython.display import display, HTML

#REferences: https://towardsdatascience.com/data-handling-using-pandas-cleaning-and-processing-3aa657dc9418
#Dataset: https://www.kaggle.com/hugomathien/soccer?

In [131]:
with sqlite3.connect('database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    player = pd.read_sql_query("SELECT * from Player",con)
    player_attributes = pd.read_sql_query("SELECT * from Player_Attributes",con)
    sequence = pd.read_sql_query("SELECT * from sqlite_sequence",con)
    team_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)

In [177]:
df = player_attributes
df.head(5)
print(len(df))

183978


## Remove unecessary columns

In [178]:
df = df.drop(columns = ['id', 'player_api_id', 'date'])

## Data overview

In [179]:
#missing Values
print(df.isna().sum())

df_rows_with_na = df[df.isna().any(axis=1)]
print("percentage of rows with missing data is ", len(df_rows_with_na)*100//len(df), "%" )

display(df[df["volleys"].isna()])

player_fifa_api_id        0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
gk_handling         

,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
446,52782,68.0,70.0,right,None,_0,60.0,50.0,60.0,74.0,...,NaN,47.0,52.0,50.0,NaN,7.0,20.0,62.0,20.0,20.0
447,52782,68.0,70.0,right,None,_0,60.0,50.0,60.0,74.0,...,NaN,47.0,52.0,50.0,NaN,7.0,20.0,62.0,20.0,20.0
448,52782,68.0,69.0,right,None,_0,60.0,50.0,60.0,74.0,...,NaN,47.0,52.0,50.0,NaN,7.0,20.0,62.0,20.0,20.0
449,52782,70.0,69.0,right,None,_0,60.0,50.0,60.0,74.0,...,NaN,47.0,52.0,50.0,NaN,7.0,20.0,62.0,20.0,20.0
450,52782,70.0,69.0,right,None,_0,60.0,50.0,70.0,74.0,...,NaN,47.0,72.0,70.0,NaN,7.0,7.0,62.0,12.0,5.0
478,33022,73.0,76.0,right,None,_0,56.0,34.0,70.0,70.0,...,NaN,68.0,76.0,75.0,NaN,5.0,23.0,71.0,23.0,23.0
479,33022,77.0,79.0,right,None,_0,68.0,34.0,80.0,72.0,...,NaN,69.0,83.0,78.0,NaN,5.0,23.0,76.0,23.0,23.0
480,33022,77.0,80.0,right,None,_0,68.0,34.0,80.0,72.0,...,NaN,69.0,83.0,78.0,NaN,5.0,23.0,76.0,23.0,23.0
481,33022,78.0,79.0,right,None,_0,72.0,34.0,80.0,72.0,...,NaN,69.0,80.0,76.0,NaN,5.0,23.0,76.0,23.0,23.0
482,33022,78.0,79.0,right,None,_0,72.0,34.0,80.0,72.0,...,NaN,69.0,80.0,76.0,NaN,5.0,10.0,76.0,10.0,7.0


## Wrong Data Values

### Categorial Variables 

In [180]:
# Print all unique values
categorial_columns = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
for c in categorial_columns:
    print(c, df[c].unique())
    
# Come up with a set of possible values
pos_values_preferred_foot = ['right', 'left']
pos_values_attacking_work =['medium', 'high', 'low']
pos_values_defensive_work = ['medium', 'high', 'low']


preferred_foot ['right' 'left' None]
attacking_work_rate ['medium' 'high' None 'low' 'None' 'le' 'norm' 'stoc' 'y']
defensive_work_rate ['medium' 'high' 'low' '_0' None '5' 'ean' 'o' '1' 'ormal' '7' '2' '8' '4'
 'tocky' '0' '3' '6' '9' 'es']


In [181]:
# Note: there are None and "None" values
def analysePossValues(df, cat_name, pos_values):
    
    valid_rows = df.loc[df[cat_name].isin(pos_values)]
    invalid_rows = df.loc[df[cat_name].isin(pos_values) == False]
    print("percentage of invalid rows is", len(invalid_rows)*100//len(df), "%")
    
    #check distribution
    valid_rows[cat_name].value_counts().plot(kind='bar')

def subWithMode(df, cat_name, pos_values):
    print('values before change\n', df[cat_name].value_counts())
    
    valid_rows = df.loc[df[cat_name].isin(pos_values)]
    value_counts = valid_rows[cat_name].value_counts()
    mode_label = value_counts.idxmax()
    print('mode label is', mode_label)
    
    df.loc[df[cat_name].isin(pos_values) == False, cat_name] = mode_label
    print('values after change\n', df[cat_name].value_counts())
    return df
    
    
    


In [182]:
df = subWithMode(df,'preferred_foot' , pos_values_preferred_foot)
df = subWithMode(df,'attacking_work_rate' , pos_values_attacking_work)
df = subWithMode(df,'defensive_work_rate' , pos_values_defensive_work)
print(df['defensive_work_rate'].value_counts())

values before change
 right    138409
left      44733
Name: preferred_foot, dtype: int64
mode label is right
values after change
 right    139245
left      44733
Name: preferred_foot, dtype: int64
values before change
 medium    125070
high       42823
low         8569
None        3639
norm         348
y            106
le           104
stoc          89
Name: attacking_work_rate, dtype: int64
mode label is medium
values after change
 medium    132586
high       42823
low         8569
Name: attacking_work_rate, dtype: int64
values before change
 medium    130846
high       27041
low        18432
_0          2394
o           1550
1            441
ormal        348
2            342
3            258
5            234
7            217
6            197
0            197
9            152
4            116
es           106
ean          104
tocky         89
8             78
Name: defensive_work_rate, dtype: int64
mode label is medium
values after change
 medium    138505
high       27041
low        

### Numerical Variables

In [183]:
# Check if all are float
id_column = "player_fifa_api_id"
numerical_columns = list(set(df.columns) - set(categorial_columns) - set([id_column]))
print(df[numerical_columns].dtypes)
display(df[numerical_columns].head(3))

balance               float64
acceleration          float64
reactions             float64
strength              float64
gk_kicking            float64
gk_reflexes           float64
sliding_tackle        float64
ball_control          float64
gk_diving             float64
volleys               float64
free_kick_accuracy    float64
curve                 float64
short_passing         float64
sprint_speed          float64
jumping               float64
gk_handling           float64
stamina               float64
positioning           float64
dribbling             float64
interceptions         float64
crossing              float64
potential             float64
standing_tackle       float64
marking               float64
long_shots            float64
agility               float64
gk_positioning        float64
overall_rating        float64
heading_accuracy      float64
finishing             float64
long_passing          float64
shot_power            float64
vision                float64
penalties 

,balance,acceleration,reactions,strength,gk_kicking,gk_reflexes,sliding_tackle,ball_control,gk_diving,volleys,...,agility,gk_positioning,overall_rating,heading_accuracy,finishing,long_passing,shot_power,vision,penalties,aggression
0,65.0,60.0,47.0,76.0,10.0,8.0,69.0,49.0,6.0,44.0,...,59.0,8.0,67.0,71.0,44.0,64.0,55.0,54.0,48.0,71.0
1,65.0,60.0,47.0,76.0,10.0,8.0,69.0,49.0,6.0,44.0,...,59.0,8.0,67.0,71.0,44.0,64.0,55.0,54.0,48.0,71.0
2,65.0,60.0,47.0,76.0,10.0,8.0,69.0,49.0,6.0,44.0,...,59.0,8.0,62.0,71.0,44.0,64.0,55.0,54.0,48.0,63.0


In [184]:
column_check = df[numerical_columns].isna().sum()
print(column_check)

balance               2713
acceleration           836
reactions              836
strength               836
gk_kicking             836
gk_reflexes            836
sliding_tackle        2713
ball_control           836
gk_diving              836
volleys               2713
free_kick_accuracy     836
curve                 2713
short_passing          836
sprint_speed           836
jumping               2713
gk_handling            836
stamina                836
positioning            836
dribbling              836
interceptions          836
crossing               836
potential              836
standing_tackle        836
marking                836
long_shots             836
agility               2713
gk_positioning         836
overall_rating         836
heading_accuracy       836
finishing              836
long_passing           836
shot_power             836
vision                2713
penalties              836
aggression             836
dtype: int64


In [185]:
# Confirm that the 836/2713 missing values belong to the same record

column_check = df[numerical_columns].isna().sum()
missing_columns_set1 = list(column_check[column_check == 836].index)
missing_columns_set2 = list(column_check[column_check == 2713].index)

df_set1 = df[missing_columns_set1]
df_set2 = df[missing_columns_set2]

df1_rows_na = df_set1[df_set1.isna().any(axis=1)]
df2_rows_na = df_set2[df_set2.isna().any(axis=1)]

print(len(df1_rows_na))
print(len(df2_rows_na))


836
2713


In [186]:
#Full outer join
rows_to_drop = df1_rows_na.merge(df2_rows_na, left_index = True, right_index=True, how="outer")
print("number of rows to drop is", len(rows_to_drop))

number of rows to drop is 2713


In [187]:
print("initial df length is ", len(df))
# df = df.drop(df1_rows_na.index)
# df = df.drop(df2_rows_na.index)
df = df.drop(rows_to_drop.index)
print("new df length is ", len(df))

#double check
column_check = df[numerical_columns].isna().sum()
print(column_check)

initial df length is  183978
new df length is  181265
balance               0
acceleration          0
reactions             0
strength              0
gk_kicking            0
gk_reflexes           0
sliding_tackle        0
ball_control          0
gk_diving             0
volleys               0
free_kick_accuracy    0
curve                 0
short_passing         0
sprint_speed          0
jumping               0
gk_handling           0
stamina               0
positioning           0
dribbling             0
interceptions         0
crossing              0
potential             0
standing_tackle       0
marking               0
long_shots            0
agility               0
gk_positioning        0
overall_rating        0
heading_accuracy      0
finishing             0
long_passing          0
shot_power            0
vision                0
penalties             0
aggression            0
dtype: int64


In [198]:
print(df['preferred_foot'].value_counts())

right    136953
left      44312
Name: preferred_foot, dtype: int64


## One-hot encode

In [199]:
#one hot encode everything!
def one_hot_encode(df, column_name):
    temp = df.copy()
    df = df.drop(column_name,axis = 1)
    df = df.join(pd.get_dummies(temp[column_name], prefix=column_name))
    return df
# df = one_hot_encode(df, col_name)

In [200]:
for col_name in categorial_columns:
    df = one_hot_encode(df, col_name)

## Final checks and saving

In [201]:
df.isna().sum()

player_fifa_api_id            0
overall_rating                0
potential                     0
crossing                      0
finishing                     0
heading_accuracy              0
short_passing                 0
volleys                       0
dribbling                     0
curve                         0
free_kick_accuracy            0
long_passing                  0
ball_control                  0
acceleration                  0
sprint_speed                  0
agility                       0
reactions                     0
balance                       0
shot_power                    0
jumping                       0
stamina                       0
strength                      0
long_shots                    0
aggression                    0
interceptions                 0
positioning                   0
vision                        0
penalties                     0
marking                       0
standing_tackle               0
sliding_tackle                0
gk_divin

In [202]:
df.dtypes

player_fifa_api_id              int64
overall_rating                float64
potential                     float64
crossing                      float64
finishing                     float64
heading_accuracy              float64
short_passing                 float64
volleys                       float64
dribbling                     float64
curve                         float64
free_kick_accuracy            float64
long_passing                  float64
ball_control                  float64
acceleration                  float64
sprint_speed                  float64
agility                       float64
reactions                     float64
balance                       float64
shot_power                    float64
jumping                       float64
stamina                       float64
strength                      float64
long_shots                    float64
aggression                    float64
interceptions                 float64
positioning                   float64
vision      

In [204]:
csv_path = "cleaned_soccer_data_v1.csv"
df.to_csv(csv_path)

## Testing

In [ ]:
# #Find KNN for replacement
# def replaceNA(df, cat_name):

#     numerical_df = df.drop(columns=['player_fifa_api_id', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])
#     print(numerical_df.isna().sum())
# #         knn = NearestNeighbors(n_neighbors=5)
# #     knn.fit(df.drop(columns=['player_fifa_api_id', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']))
# #     knn.kneighbors(X[0], return_distance=False)

# replaceNA(df, 'attacking_work_rate')

In [71]:
df_test = pd.DataFrame([[1, True, 'dsfasd', 51.314],
                   [51, False, '56345', 56.1234]],
                  columns=['col1', 'col2', 'col3', 'col4'])

res = df_test.dtypes

print(res)

col1      int64
col2       bool
col3     object
col4    float64
dtype: object
